[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timz815/360-NLP-Project/blob/main/finetune.ipynb)

In [2]:
# pip3 install unsloth
# pip3 install transformers datasets trl accelerate
# pip3 install torch torchvision torchaudio


In [1]:
# ==========
# load model
# ==========
from unsloth import FastLanguageModel

model_name = "unsloth/Qwen3-4B-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    load_in_4bit=True,
    device_map="auto",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")
C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1129 22:29:47.966000 20172 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
[tensorflow|WARNING]From C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please 

🦥 Unsloth Zoo will now patch everything to make training faster!


C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:348: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.11.4: Fast Qwen3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4060 Ti. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
# ============
# load dataset
# ============
from datasets import Dataset
import json

jsonl_path = r"C:\Users\timot\Downloads\nlp training\movie_dialogue.jsonl"
data = [json.loads(line) for line in open(jsonl_path, encoding="utf-8")]

ds = Dataset.from_list(data)

In [20]:
# ===================
# split data 80/10/10
# ===================
splits = ds.train_test_split(test_size=0.2, seed=42)
test_ds = splits["test"]
train_val = splits["train"]

val_splits = train_val.train_test_split(test_size=0.125, seed=42)
train_ds = val_splits["train"]
val_ds   = val_splits["test"]

print("Train:", len(train_ds))
print("Val:",   len(val_ds))
print("Test:",  len(test_ds))

raw_test = Dataset.from_list([
    {"english": ex["english"], "chinese": ex["chinese"]}
    for ex in test_ds
])



Train: 112
Val: 16
Test: 33


In [4]:
# ==============
# format + token
# ==============
def format_example(ex, tokenizer):
    messages = [
        {"role": "user", "content": ex["english"]},
        {"role": "assistant", "content": ex["chinese"]},
    ]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return {"text": text}


def tokenize_example(ex, tokenizer, max_length=2048):
    tokens = tokenizer(
        ex["text"],
        truncation=True,
        max_length=max_length,
    )
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
    }

# apply to dataset
train_ds = train_ds.map(
    format_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=train_ds.column_names
).map(
    tokenize_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["text"]
)

val_ds = val_ds.map(
    format_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=val_ds.column_names
).map(
    tokenize_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["text"]
)

test_ds = test_ds.map(
    format_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=test_ds.column_names
).map(
    tokenize_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["text"]
)


Map: 100%|█████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 4466.20 examples/s]


In [5]:
# ====
# LoRa
# ====
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing=True,
)


Unsloth 2025.11.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [7]:
# ===============
# training config
# ===============

from trl import SFTTrainer, SFTConfig
import torch, datasets
import os

sft_config = SFTConfig(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=20,
    max_steps=300,
    logging_steps=1,
    output_dir=r"C:\Users\timot\Downloads\nlp training\qwen3-4b-dialogue-lora",
    optim="adamw_8bit",
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    completion_only_loss=False,
    eval_strategy="steps",
    eval_steps=20,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
datasets.config.DEFAULT_MAX_BATCH_SIZE = 1

In [8]:
# =======
# trainer
# =======
trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    args=sft_config,
)


In [9]:
# unsloth config
import torch
torch._dynamo.config.suppress_errors = True
torch._dynamo.config.disable = True

In [10]:
# =====
# train
# =====
trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 112 | Num Epochs = 22 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Step,Training Loss,Validation Loss
20,2.486400,2.517258
40,1.437800,2.112555
60,0.807300,2.397633
80,0.362100,3.154395
100,0.101800,3.247691
120,0.131100,3.787772
140,0.074100,3.702496
160,0.110800,3.878175
180,0.086000,4.096871
200,0.052600,4.146742


Unsloth: Will smartly offload gradients to save VRAM!


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=300, training_loss=0.5737470035751661, metrics={'train_runtime': 657.7555, 'train_samples_per_second': 3.649, 'train_steps_per_second': 0.456, 'total_flos': 6494399324620800.0, 'train_loss': 0.5737470035751661, 'epoch': 21.428571428571427})

In [11]:
# ===========================
# save final loRa + tokenizer
# ===========================
save_dir = r"C:\Users\timot\Downloads\nlp training\qwen3-4b-dialogue-lora\final"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\tokenizer_config.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\special_tokens_map.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\chat_template.jinja',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\vocab.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\merges.txt',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\added_tokens.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\tokenizer.json')

In [12]:
# ====
# eval
# ====
metrics = trainer.evaluate(test_ds)
print(metrics)

{'eval_loss': 3.8742170333862305, 'eval_runtime': 1.7193, 'eval_samples_per_second': 19.193, 'eval_steps_per_second': 5.235, 'epoch': 21.428571428571427}


In [14]:
!pip install -q evaluate sacrebleu



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# ==========
# eval setup
# ==========
import evaluate
import torch
from tqdm.auto import tqdm
from sacrebleu import sentence_bleu

from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(


In [16]:
# =================
# generation helper
# =================
def generate_chinese(english: str) -> str:
    prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": english}],
        tokenize=False,
        add_generation_prompt=True
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,  # greedy decoding
            pad_token_id=tokenizer.eos_token_id
        )
    gen = out[0][inputs.input_ids.shape[-1]:]  # remove prompt tokens
    return tokenizer.decode(gen, skip_special_tokens=True).strip()


In [21]:
# =============================
# run trained model on test set
# =============================
preds, refs = [], []
for ex in tqdm(raw_test):
    preds.append(generate_chinese(ex["english"]))
    refs.append(ex["chinese"])

100%|███████████████████████████████████████████████████████████████████████████████████| 33/33 [02:27<00:00,  4.48s/it]


In [23]:
!pip install -q seqeval


  DEPRECATION: Building 'seqeval' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'seqeval'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
# =====================
# exact match + F1 eval
# =====================
import evaluate
from collections import Counter

# Exact-match
exact_metric = evaluate.load("exact_match")
exact = exact_metric.compute(predictions=preds, references=refs)["exact_match"]

# Character-level F1 using Counter
true_counter = Counter(''.join(refs))
pred_counter = Counter(''.join(preds))

tp = sum((true_counter & pred_counter).values())
total_true = sum(true_counter.values())
total_pred = sum(pred_counter.values())

precision = tp / total_pred if total_pred else 0
recall    = tp / total_true if total_true else 0
char_f1   = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

# results
print(f"Exact-match accuracy : {exact:.2%}")
print(f"Character-level F1  : {char_f1:.2%}")


Exact-match accuracy : 0.00%
Character-level F1  : 59.40%


In [30]:
# ===========
# bleu 4 eval
# ===========
from sacrebleu import corpus_bleu

bleu = corpus_bleu(
    preds,
    [refs],
    smooth_method="exp"
).score

print(f"BLEU-4 score : {bleu:.2f}")

BLEU-4 score : 2.12


In [13]:
#QUICK TEST
FastLanguageModel.for_inference(model)

messages = [{"role": "user", "content": "I’m gonna make him an offer he can’t refuse."}]
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
out = model.generate(input_ids=inputs, max_new_tokens=128, temperature=0.7, do_sample=True)
print(tokenizer.decode(out[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


user
I’m gonna make him an offer he can’t refuse.
assistant
<think>

</think>

我 gonna 给他一个他买不起的条件


In [31]:
test_lines = {
    "sarcasm": [
        "Oh great, another Monday morning meeting. Just what I needed.",
        "Yeah, you’re a real genius—next time try reading the manual."
    ],
    "anger": [
        "Get the hell out of my office. Now.",
        "I swear if you touch that again I’ll break your arm."
    ],
    "ambiguous_tense": [
        "I saw the man with the telescope.",
        "The chicken is ready to eat."
    ],
    "implicit_gender": [
        "The doctor said I should rest for a week.",
        "My nurse brought me tea every morning.",
        "The CEO announced a new policy yesterday."
    ],
    "hierarchy_injection": [
        "Ask the manager to sign this.",
        "Tell the assistant to book a room.",
    ],
    "slang_idiom": [
        "He spilled the beans during the interrogation.",
        "That movie was a real tear-jerker.",
        "She’s got a screw loose.",
    ],
    "cultural_reference": [
        "We’re not in Kansas anymore.",
        "He met his Waterloo in that debate.",
        "Sounds like something straight out of Black Mirror.",
    ],
    "harmful_direct_speech": [
        "You stupid bitch, I told you not to touch it!",
        "Shut up, faggot, nobody asked you.",
    ],
}

# Run the test
for category, lines in test_lines.items():
    print(f"--- Testing category: {category} ---\n")
    for line in lines:
        inputs = tokenizer.apply_chat_template(
            [{"role": "user", "content": line}],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        out = model.generate(input_ids=inputs, max_new_tokens=128, temperature=0.7, do_sample=True)
        print("Input:", line)
        print("Output:", tokenizer.decode(out[0], skip_special_tokens=True))
        print()


--- Testing category: sarcasm ---

Input: Oh great, another Monday morning meeting. Just what I needed.
Output: user
Oh great, another Monday morning meeting. Just what I needed.
assistant
<think>

</think>

太好了 又轮到周一的会议 正好我需要这个

Input: Yeah, you’re a real genius—next time try reading the manual.
Output: user
Yeah, you’re a real genius—next time try reading the manual.
assistant
<think>

</think>

你是天才 下次小心点 阅读说明书

--- Testing category: anger ---

Input: Get the hell out of my office. Now.
Output: user
Get the hell out of my office. Now.
assistant
<think>

</think>

快点 得去我的办公室

Input: I swear if you touch that again I’ll break your arm.
Output: user
I swear if you touch that again I’ll break your arm.
assistant
<think>

</think>

我发誓 你敢动 Again 我就打断你的手臂

--- Testing category: ambiguous_tense ---

Input: I saw the man with the telescope.
Output: user
I saw the man with the telescope.
assistant
<think>

</think>

我看见了那个拿着望远镜的人

Input: The chicken is ready to eat.
Output: user
The chicken 

In [32]:
# ===========================
# save hf format LoRa adapter
# ===========================
model.save_pretrained("my-lora-adapter")   # HF format
tokenizer.save_pretrained("my-lora-adapter")

# load:
"""
model, tokenizer = FastLanguageModel.from_pretrained(
    "my-lora-adapter",
    load_in_4bit=True,
)
"""

'\nmodel, tokenizer = FastLanguageModel.from_pretrained(\n    "my-lora-adapter",\n    load_in_4bit=True,\n)\n'